In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


import numpy as np
from numpy.random import seed, randint

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [6]:
import os
os.path.abspath(os.getcwd())


'/'

In [27]:
X = np.load('/project/XSound.npy')
Y = tf.keras.utils.to_categorical(np.load('/project/YSound.npy'))
#Y = np.load('/project/YSound.npy')
print(X.shape)
print(Y.shape)
print(Y)

(15737, 62, 65, 1)
(15737, 4)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


In [28]:
X_, X_test, Y_, Y_test = train_test_split(X, Y, train_size=.8, random_state=504)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_, Y_, train_size=.75, random_state=504)
#X_train, y_train = train_test_split(X_train, Y_train)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(X_valid.shape)
print(Y_valid.shape)

(9441, 62, 65, 1)
(9441, 4)
(3148, 62, 65, 1)
(3148, 4)
(3148, 62, 65, 1)
(3148, 4)


In [46]:
print(X_train[0].shape)

(62, 65, 1)


In [56]:
es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)

cnn = tf.keras.models.Sequential([
    layers.Conv2D(16, 3, activation='relu', padding='same', input_shape=X_train[0].shape),
    layers.MaxPooling2D((3)),
    layers.SpatialDropout2D(.4),
    layers.AveragePooling2D(),
    #layers.Conv2D(32, 3, activation='relu', padding='same'),
    #layers.SpatialDropout2D(.4),
    #layers.AveragePooling2D(),
    #layers.Conv2D(120, 3, activation='relu', padding='same'),
    #layers.SpatialDropout2D(.4),
    layers.Flatten(),
    layers.Dense(units=84, activation='relu'),
    layers.Dropout(.4),
    layers.Dense(units=4, activation='softmax')
])

cnn.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), metrics = ['accuracy'])
cnn.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 62, 65, 16)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 20, 21, 16)        0         
_________________________________________________________________
spatial_dropout2d_26 (Spatia (None, 20, 21, 16)        0         
_________________________________________________________________
average_pooling2d_20 (Averag (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 10, 10, 32)        4640      
_________________________________________________________________
spatial_dropout2d_27 (Spatia (None, 10, 10, 32)        0         
_________________________________________________________________
average_pooling2d_21 (Averag (None, 5, 5, 32)        

In [50]:
history = cnn.fit(X_train, Y_train, 
                    epochs = 100, 
                    validation_data=(X_valid, Y_valid),
                    callbacks=[es])

Train on 9441 samples, validate on 3148 samples
Epoch 1/100
9441/9441 [==============================] - 859s 91ms/sample - loss: 1.3866 - accuracy: 0.2508 - val_loss: 1.3977 - val_accuracy: 0.2367
Epoch 2/100
9441/9441 [==============================] - 1s 107us/sample - loss: 1.4045 - accuracy: 0.2572 - val_loss: 1.3886 - val_accuracy: 0.2564
Epoch 3/100
9441/9441 [==============================] - 1s 102us/sample - loss: 1.3913 - accuracy: 0.2557 - val_loss: 1.3877 - val_accuracy: 0.2570
Epoch 4/100
9441/9441 [==============================] - 1s 118us/sample - loss: 1.3900 - accuracy: 0.2519 - val_loss: 1.3873 - val_accuracy: 0.2564
Epoch 5/100
9441/9441 [==============================] - 1s 105us/sample - loss: 1.3909 - accuracy: 0.2568 - val_loss: 1.3894 - val_accuracy: 0.2367
Epoch 6/100
9441/9441 [==============================] - 1s 101us/sample - loss: 1.3917 - accuracy: 0.2491 - val_loss: 1.3882 - val_accuracy: 0.2351
Epoch 7/100
9441/9441 [==============================] - 

In [55]:
train_eval = cnn.evaluate(X_train[:9441], Y_train[:9441])
test_eval = cnn.evaluate(X_test, Y_test)
print("On train data: Loss = {loss:.4f}, accuracy = {accuracy:.4f}".format(loss=train_eval[0], accuracy=train_eval[1]))
print("On test data: Loss = {loss:.4f}, accuracy = {accuracy:.4f}".format(loss=test_eval[0], accuracy=test_eval[1]))
print("Error on test data: {p:.2f}%".format(p = 100*(1-test_eval[1])))

3148/3148 [==============================] - 0s 47us/sample - loss: 1.4800 - accuracy: 0.2630
On train data: Loss = 1.5041, accuracy = 0.2546
On test data: Loss = 1.4800, accuracy = 0.2630
Error on test data: 73.70%
